In [61]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [12]:
path_items = '../data/processed/df_items_clean.parquet'
path_genres = '../data/processed/df_genres_dummies.parquet'
path_games = '../data/processed/df_games_clean.parquet'
path_reviews = '../data/processed/df_reviews_clean.parquet'


df_items = pd.read_parquet(path_items)
df_genres = pd.read_parquet(path_genres)
df_games = pd.read_parquet(path_games)
df_reviews = pd.read_parquet(path_reviews)

df_items['game_id'] = df_items['game_id'].astype('Int64')
df_genres['game_id'] = df_genres['game_id'].astype('Int64')
df_games['game_id'] = df_games['game_id'].astype('Int64')
df_reviews['game_id'] = df_reviews['game_id'].astype('Int64')
df_games = pd.read_parquet(path_games)

Function 1:

In [ ]:

save_path = '../data/processed/'

playtime_genre = pd.merge(df_items, df_games, on='game_id')
playtime_genre = pd.merge(playtime_genre, df_genres, on='game_id')
playtime_genre.to_parquet('playtime_genre.parquet')

In [67]:
save_path = '../data/processed/'

# Ensure 'playtime_forever' is included in the columns of df_items
columns_items_required = ['game_id', 'playtime_forever']  # Add more columns if necessary
columns_games_required = ['game_id', 'release_year']  # Adjust as needed
columns_genres_required = df_genres.columns  # All genre columns

# Select only the necessary columns from each dataframe
df_items_reduced = df_items[columns_items_required]
df_games_reduced = df_games[columns_games_required]
df_genres_reduced = df_genres[columns_genres_required]

# Perform the joins
df_combined = pd.merge(df_items_reduced, df_games_reduced, on='game_id')
df_combined = pd.merge(df_combined, df_genres_reduced, on='game_id')

# Save the combined dataframe
df_combined.to_parquet(save_path + 'playtime_genre_combined.parquet', compression='snappy')


In [64]:
def playtime_genre(genre: str):
    # Load the combined dataset
    df_combined = pd.read_parquet(save_path + 'playtime_genre_combined.parquet')

    # Convert the genre to lowercase for comparison
    genre_lower = genre.lower()

    # Find the corresponding column for the genre
    genre_column = next((col for col in df_combined.columns if col.lower() == genre_lower), None)
    
    if not genre_column:
        return {"error": f"Genre '{genre}' not found"}

    # Filter by the specified genre
    genre_df = df_combined[df_combined[genre_column] == 1]

    # Group by release year and sum the playtime
    playtime_by_year = genre_df.groupby('release_year')['playtime_forever'].sum()

    # Find the year with the maximum playtime
    max_playtime_year = playtime_by_year.idxmax()

    # Return the result
    return {"Year with most hours played for Genre {}".format(genre): int(max_playtime_year)}


In [65]:

result_1 = playtime_genre("action")
result_2 = playtime_genre("adventure")
result_3 = playtime_genre("strategy")

print(result_1)
print(result_2)
print(result_3)


{'Year with most hours played for Genre action': 2012}
{'Year with most hours played for Genre adventure': 2006}
{'Year with most hours played for Genre strategy': 2012}


In [23]:
df_combinado.head()

,game_id,playtime_forever,release_year,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,...,Photo Editing,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing
0,10,6.0,2000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,10,0.0,2000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,10,0.0,2000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,10,93.0,2000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,10,108.0,2000,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


Function 2:

In [37]:
# Assuming df_items, df_games, and df_genres are already loaded

# Select necessary columns
columns_items_required = ['game_id', 'user_id', 'playtime_forever']
columns_games_required = ['game_id', 'release_year']  # Include release_year
columns_genres_required = df_genres.columns  # All genre columns

df_items_reduced = df_items[columns_items_required]
df_games_reduced = df_games[columns_games_required]
df_genres_reduced = df_genres[columns_genres_required]

# Convert playtime_forever to integer
df_items_reduced['playtime_forever'] = df_items_reduced['playtime_forever'].fillna(0).astype(int)

# Perform the joins
df_combined = pd.merge(df_items_reduced, df_games_reduced, on='game_id')
df_combined = pd.merge(df_combined, df_genres_reduced, on='game_id')

# Save the dataframe combined
save_path = '../data/processed/'
df_combined.to_parquet(save_path + 'user_genre_combined.parquet', compression='snappy')



C:\Users\sarmi\AppData\Local\Temp\ipykernel_10900\1714459854.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_items_reduced['playtime_forever'] = df_items_reduced['playtime_forever'].fillna(0).astype(int)


In [38]:
def user_for_genre(genre: str):
    # Load the combined dataset
    df_combined = pd.read_parquet(save_path + 'user_genre_combined.parquet')

    # Convert the genre to lowercase for comparison
    genre_lower = genre.lower()

    # Find the corresponding column for the genre
    genre_column = next((col for col in df_combined.columns if col.lower() == genre_lower), None)
    
    if not genre_column:
        return {"error": f"Genre '{genre}' not found"}

    # Filter by the specified genre
    genre_df = df_combined[df_combined[genre_column] == 1]

    # Group by user_id and sum the playtime
    user_playtime = genre_df.groupby('user_id')['playtime_forever'].sum()
    top_user = user_playtime.idxmax()

    # Analyze playtime by year for the top user
    playtime_by_year = genre_df[genre_df['user_id'] == top_user].groupby('release_year')['playtime_forever'].sum().reset_index()
    playtime_by_year = playtime_by_year[playtime_by_year['playtime_forever'] > 0]
    playtime_by_year = playtime_by_year.sort_values('release_year', ascending=False)

    # Return the result
    return {
        "User with most hours played for Genre {}".format(genre): str(top_user),
        "Hours Played": playtime_by_year.to_dict(orient='records')
    }


In [39]:
# Example genres for testing. Replace these with genres from your dataset.
test_genres = ["action", "adventure", "strategy"]

# Testing the function with each genre
for genre in test_genres:
    result = user_for_genre(genre)
    print(f"Results for genre '{genre}':")
    print(result)
    print("\n")


Results for genre 'action':
{'User with most hours played for Genre action': 'Sp3ctre', 'Hours Played': [{'release_year': 2017, 'playtime_forever': 43327}, {'release_year': 2016, 'playtime_forever': 29917}, {'release_year': 2015, 'playtime_forever': 394313}, {'release_year': 2014, 'playtime_forever': 132864}, {'release_year': 2013, 'playtime_forever': 122158}, {'release_year': 2012, 'playtime_forever': 384238}, {'release_year': 2011, 'playtime_forever': 155459}, {'release_year': 2010, 'playtime_forever': 82766}, {'release_year': 2009, 'playtime_forever': 108771}, {'release_year': 2008, 'playtime_forever': 8533}, {'release_year': 2007, 'playtime_forever': 112794}, {'release_year': 2006, 'playtime_forever': 90253}, {'release_year': 2005, 'playtime_forever': 21339}, {'release_year': 2004, 'playtime_forever': 127411}, {'release_year': 2003, 'playtime_forever': 7744}, {'release_year': 2002, 'playtime_forever': 238}, {'release_year': 2001, 'playtime_forever': 250}, {'release_year': 2000, 'pl

**Function 3:**

In [66]:
# Select necessary columns
columns_games_required = ['game_id', 'release_year', 'app_name']
columns_reviews_required = ['game_id', 'recommend']

df_games_reduced = df_games[columns_games_required]
df_reviews_reduced = df_reviews[columns_reviews_required]

# Perform the join
df_combined_reviews = pd.merge(df_games_reduced, df_reviews_reduced, on='game_id')

# Save the combined dataframe
save_path = '../data/processed/'
df_combined_reviews.to_parquet(save_path + 'games_reviews_combined.parquet', compression='snappy')


In [48]:
def users_recommend(year: int):
    # Load the combined dataset
    df_combined_reviews = pd.read_parquet(save_path + 'games_reviews_combined.parquet')

    # Filter for games released in the specified year
    year_games = df_combined_reviews[df_combined_reviews['release_year'] == year]

    # Check if there are games for the specified year
    if year_games.empty:
        return {"message": f"No data available for the year {year}"}

    # Filter for recommended games
    recommended_reviews = year_games[year_games['recommend'] == True]

    # Identify top 3 recommended games
    top_games = recommended_reviews['app_name'].value_counts().head(3).index.tolist()
    top_games_ranked = [{"Rank {}".format(i + 1): game} for i, game in enumerate(top_games)]

    if not top_games_ranked:
        return {"message": f"No recommended games found for the year {year}"}

    return {"Top recommended games for the year {}".format(year): top_games_ranked}



In [49]:
# Testing the function with years that are present in the dataset
test_years = [2017, 2016, 2015]  # Replace with years you want to test

for year in test_years:
    result = users_recommend(year)
    print(f"Results for the year {year}:")
    print(result)
    print("\n")


Results for the year 2017:
{'Top recommended games for the year 2017': [{'Rank 1': 'Unturned'}, {'Rank 2': 'Robocraft'}, {'Rank 3': 'ARK: Survival Evolved'}]}


Results for the year 2016:
{'Top recommended games for the year 2016': [{'Rank 1': 'Starbound'}, {'Rank 2': 'Heroes & Generals'}, {'Rank 3': "Don't Starve Together"}]}


Results for the year 2015:
{'Top recommended games for the year 2015': [{'Rank 1': 'Grand Theft Auto V'}, {'Rank 2': 'Rocket League®'}, {'Rank 3': 'Undertale'}]}




In [41]:
df_combined_reviews.head()

,game_id,release_year,app_name,recommend
0,282010,1997,Carmageddon Max Pack,True
1,70,1998,Half-Life,True
2,70,1998,Half-Life,True
3,70,1998,Half-Life,True
4,70,1998,Half-Life,True


Function 4: 

In [62]:
# Select necessary columns
columns_games_required = ['game_id', 'release_year', 'developer']
columns_reviews_required = ['game_id', 'recommend']

df_games_reduced = df_games[columns_games_required]
df_reviews_reduced = df_reviews[columns_reviews_required]

# Perform the join
df_combined_reviews_developer = pd.merge(df_games_reduced, df_reviews_reduced, on='game_id')

# Save the combined dataframe
save_path = '../data/processed/'
df_combined_reviews_developer.to_parquet(save_path + 'games_reviews_developer_combined.parquet', compression='snappy')


In [51]:
def users_worst_developer(year: int):
    # Load the combined dataset
    df_combined_reviews_developer = pd.read_parquet(save_path + 'games_reviews_developer_combined.parquet')

    # Filter for games released in the specified year
    year_games = df_combined_reviews_developer[df_combined_reviews_developer['release_year'] == year]

    # Check if there are games for the specified year
    if year_games.empty:
        return {"message": f"No data available for the year {year}"}

    # Filter for negative reviews
    negative_reviews = year_games[year_games['recommend'] == False]

    # Identify top 3 developers with the most negative reviews
    worst_developers = negative_reviews['developer'].value_counts().nsmallest(3).index.tolist()
    worst_developers_ranked = [{"Rank {}".format(i + 1): dev} for i, dev in enumerate(worst_developers)]

    if not worst_developers_ranked:
        return {"message": f"No negative reviews found for developers in the year {year}"}

    return {"Top 3 developers with the most negative reviews for the year {}".format(year): worst_developers_ranked}

In [52]:
# Example years for testing. Replace with the years you wish to test.
test_years = [2017, 2016, 2015]  # Adjust these years based on your dataset's data

# Test the function with each year
for year in test_years:
    result = users_worst_developer(year)
    print(f"Results for the year {year}:")
    print(result)
    print("\n")


Results for the year 2017:
{'Top 3 developers with the most negative reviews for the year 2017': [{'Rank 1': 'Robot Entertainment'}, {'Rank 2': 'Mojang'}, {'Rank 3': 'oddonegames'}]}


Results for the year 2016:
{'Top 3 developers with the most negative reviews for the year 2016': [{'Rank 1': 'ConcernedApe'}, {'Rank 2': 'Chad Close,Kyle Weik'}, {'Rank 3': 'WTFOMGames'}]}


Results for the year 2015:
{'Top 3 developers with the most negative reviews for the year 2015': [{'Rank 1': '3000AD'}, {'Rank 2': 'Paradox Development Studio'}, {'Rank 3': 'AGC'}]}




Function 5: 

In [53]:
# Select necessary columns
columns_games_required = ['game_id', 'developer']
columns_reviews_required = ['game_id', 'sentiment_analysis']

df_games_reduced = df_games[columns_games_required]
df_reviews_reduced = df_reviews[columns_reviews_required]

# Perform the join
df_combined_sentiment = pd.merge(df_games_reduced, df_reviews_reduced, on='game_id')

# Save the combined dataframe
save_path = '../data/processed/'
df_combined_sentiment.to_parquet(save_path + 'games_sentiment_combined.parquet', compression='snappy')


In [55]:
def sentiment_analysis(developer: str):
    # Load the combined dataset
    df_combined_sentiment = pd.read_parquet(save_path + 'games_sentiment_combined.parquet')

    # Convert the developer name to lowercase for comparison
    developer_lower = developer.lower()

    # Find games developed by the specified developer
    matched_developer = df_combined_sentiment['developer'].str.lower().eq(developer_lower)
    if not matched_developer.any():
        return {"error": f"Developer '{developer}' not found"}

    # Filter reviews for the specified developer
    dev_reviews = df_combined_sentiment[matched_developer]
    if dev_reviews.empty:
        return {"message": "No reviews found for developer {}".format(developer)}

    # Perform sentiment analysis count
    sentiment_count = dev_reviews['sentiment_analysis'].value_counts().rename({0: 'Negative', 1: 'Neutral', 2: 'Positive'})

    return {"Sentiment analysis for developer {}".format(developer): sentiment_count.to_dict()}


In [59]:
# Example developers for testing. Replace with developers from your dataset.
test_developers = ["Valve", "Stainless Games Ltd", "Test Missing"]

# Test the function with each developer
for developer in test_developers:
    result = sentiment_analysis(developer)
    print(f"Results for developer '{developer}':")
    print(result)
    print("\n")


Results for developer 'Valve':
{'Sentiment analysis for developer Valve': {'Positive': 5479, 'Neutral': 2597, 'Negative': 1621}}


Results for developer 'Stainless Games Ltd':
{'Sentiment analysis for developer Stainless Games Ltd': {'Positive': 2, 'Neutral': 1}}


Results for developer 'Test Missing':
{'error': "Developer 'Test Missing' not found"}




In [54]:
df_combined_sentiment.head()

,game_id,developer,sentiment_analysis
0,282010,Stainless Games Ltd,1
1,70,Valve,0
2,70,Valve,0
3,70,Valve,0
4,70,Valve,2


ML Function: 



In [68]:
# Step 1 & 2: Combine df_games and df_genres and apply StandardScaler
df_merged = df_games.merge(df_genres, on='game_id', how='left')
features = ['release_year'] + list(df_genres.columns[1:])
scaler = StandardScaler()
df_merged['release_year'] = scaler.fit_transform(df_merged[['release_year']])

# Step 3: Merge with app_name
df_final = df_merged[['game_id'] + features]
df_final = df_final.merge(df_games[['game_id', 'app_name']], on='game_id', how='left')

# Step 4: Save the pre-processed data
preprocessed_data_path = save_path + 'preprocessed_for_recommendation.parquet'
df_final.to_parquet(preprocessed_data_path, compression='snappy')


In [69]:
df_final.head()


,game_id,release_year,Accounting,Action,Adventure,Animation & Modeling,Audio Production,Casual,Design & Illustration,Early Access,...,RPG,Racing,Simulation,Software Training,Sports,Strategy,Utilities,Video Production,Web Publishing,app_name
0,761140,0.963105,0,1,0,0,0,1,0,0,...,0,0,1,0,0,1,0,0,0,Lost Summoner Kitty
1,643980,0.963105,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,0,0,Ironbound
2,670290,0.657395,0,0,0,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,Real Pool 3D - Poolians
3,767400,0.657395,0,1,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,弹炸人2222
4,772540,0.963105,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,Battle Royale Trainer


In [11]:
def recomend_game_sampled(game_id, top_n=5):
    # Load the pre-processed data
    df_final = pd.read_parquet(preprocessed_data_path)

    # Perform sampling
    df_sampled = df_final.sample(frac=0.5, random_state=42)

    # Calculate similarity matrix
    similarity_matrix = cosine_similarity(df_sampled[features].fillna(0))
    similarity_matrix = np.nan_to_num(similarity_matrix)

    # Rest of your function logic...


{'Usuario con más horas jugadas para Género': 'Education', 'Usuario': 'tsunamitad', 'Horas jugadas': [{'Año': 2011, 'Horas': 600068.0}, {'Año': 2013, 'Horas': 526.0}, {'Año': 2010, 'Horas': 430.0}]}
